# Corona prepping using Finnish data regression using OLS regression and the Potential for Change Index (PCI)

## Main question: at this point we're interested in one single regression, i.e. __what predicts whether people do maskless contacts with non-householders__

[Research Document](https://docs.google.com/document/d/1iLciHcvVvf8QwFS7wiyNBevpD1B9yDRqMlM4_oCcVcA/edit?usp=sharing)

[Questions codebook](https://docs.google.com/document/d/1YZVCP1UNxnNLAK2kYDfA9Y98leTZYurZD-d8iByhdi0/edit?usp=sharing)

[Method of delivery](https://docs.google.com/document/d/1G1JT9JUJrTK3aaXXuRawYACJaGNxU7mcXL9i-d8eKXY/edit)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import session_info

from ml_class import (plot_cv_indices,
                      plot_decision_boundary,
                     plot_learning_curve,
                     multi_roc_auc_plot,
                     dict_of_models,
                     RFE_opt_rf,
                     make_confusion_matrix,
                     summary_performance_metrics_classification)

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, GroupKFold, GroupShuffleSplit, RepeatedStratifiedKFold, RepeatedKFold
# from sklearn.impute import IterativeImputer

ModuleNotFoundError: No module named 'session_info'

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy import stats
# from sklearn.feature_selection import RFE, RFECV

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, cross_validate
# from sklearn.ensemble import BaggingClassifier, BaggingRegressor
# from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
import pingouin as pg
import statsmodels.api as sm

In [ ]:
_ = sns.set_style("whitegrid")

### Virtual Environments and Packages

In [ ]:
session_info.show(req_file_name="corona_preppers-requirements.txt",
      write_req_file=False) #add write_req_file=True to function to get requirements.txt file of packages used

### Read in data, show info and data head

In [ ]:
df = pd.read_csv("data/shield_gjames_21-09-20_prepped.csv").drop("Unnamed: 0", axis=1)

In [ ]:
df.head()

In [ ]:
sdt_columns = df.filter(regex="sdt").columns.tolist()

In [ ]:
drop_sdt = True
if drop_sdt:
    df=df.drop(sdt_columns, axis=1)

In [ ]:
df.shape

### Specify the feature list, grouping variable, and specify the grouping variable as a categorical variable

In [ ]:
target = "intention_behavior_composite"

In [ ]:
df[target] = (df[target] - 10) * -1

In [ ]:
features_list = df.filter(regex="^automaticity|attitude|^norms|^risk|^effective").columns.tolist()

## EDA on the target
Check the amount of samples in the target

In [ ]:
_ = sns.violinplot(data=df[[target]].melt(), 
                    x="variable", 
                    y="value"
               )
_ = sns.stripplot(data=df[[target]].melt(), 
                    x="variable", 
                    y="value",
                  edgecolor='white',
                  linewidth=0.5
               )

In [ ]:
pd.crosstab(df["demographic_gender"], df["demographic_age"])

In [ ]:
target_df = df[target]
target_df.describe().to_frame().T

In [ ]:
_ = plt.figure(figsize=(20, 5))
_ = sns.countplot(x=target_df)
_ = plt.xticks(rotation=90)

In [ ]:
df = (df[["demographic_age", "demographic_higher_education"] + features_list + [target]]
#  .drop(drop_list, axis=1)
#  .assign(target = target_df)
#       .dropna(axis=0)
)

In [ ]:
df.info()

In [ ]:
display(df[target].value_counts().head().to_frame()), df.shape[0], df[target].value_counts().head().sum()

## Correlations between features and target

In [ ]:
X = df[features_list]
y = df[target]

In [ ]:
corrs_df = pd.DataFrame()

for column in features_list:
    temp_corr_df = (pg.corr(x=X.loc[:, column], y=y, method="pearson")
                    .reset_index()
                    .rename(columns={"index": "type"})
                    .assign(**{"feature": column})
                    .set_index("feature")
                   )
    corrs_df = pd.concat([corrs_df, temp_corr_df])
    

In [ ]:
corrs_df.sort_values("r")

In [ ]:
neg_corrs_features = corrs_df[corrs_df["r"] < 0].index.tolist()

In [ ]:
neg_corrs_features

In [ ]:
for feature in neg_corrs_features:
    _ = sns.lmplot(data=df, 
               x=target, 
               y=feature, 
               hue="demographic_age",
              legend=True)

## Multivariate Linear Regression

In [ ]:
mod = sm.OLS(endog=y, exog=X)
res = mod.fit()
display(res.summary())

## Multiple univariate regressions

In [ ]:
mod = sm.OLS(endog=y, exog=X[[neg_corrs_features[0]]], hasconst=False)
res = mod.fit(method="qr")
res.summary()

In [ ]:
model = LinearRegression(fit_intercept=True,
                        copy_X=True,
                        n_jobs=None,
                        positive=False)

reg = model.fit(X[[neg_corrs_features[0]]], y)

reg.coef_

In [ ]:
model = LinearRegression(fit_intercept=False,
                        copy_X=True,
                        n_jobs=None,
                        positive=False)

reg = model.fit(X[[neg_corrs_features[0]]], y)

reg.coef_

In [ ]:
_ = plt.scatter(X[[neg_corrs_features[0]]], y)

In [ ]:
all_coefs_df = pd.DataFrame()
for feature in features_list:
    mod = sm.OLS(endog=y, exog=sm.add_constant(X[[feature]]))
    res = mod.fit()
    coef_df = pd.read_html(res.summary().tables[1].as_html(),header=0,index_col=0)[0]
    coef_df = coef_df.assign(**{"rsquared": res.rsquared,
                                "rsquared_adj": res.rsquared_adj})
    all_coefs_df = pd.concat([all_coefs_df, coef_df])

In [ ]:
all_coefs_df.drop("const")

In [ ]:
top_feature = all_coefs_df.drop("const").sort_values("rsquared_adj").tail(1).iloc[0].name

In [ ]:
_ = sns.lmplot(data=df, 
               x=target, 
               y=top_feature, 
               hue="demographic_age",
              legend=True)

In [ ]:
ax = sns.jointplot(data=df, 
                  x=target, 
                  y=top_feature, 
                  hue="demographic_age",
                  # kind="reg",
                   legend=True
                 )
# _ = ax._legend.remove()

In [ ]:
all_coefs_df.drop("const").describe()

In [ ]:
_ = sns.boxplot(data=all_coefs_df[["rsquared_adj", "P>|t|"]].drop("const").melt(),
                x="variable", y="value")
_ = plt.axhline(y=0.05, c="grey", ls="--")

In [ ]:
mod = sm.OLS(endog=y, exog=X[[top_feature]])
res = mod.fit()

In [ ]:
y_pred = res.predict(exog = X[[top_feature]])

df_test = pd.DataFrame({"y_pred": y_pred, target: y})

user_ids_first = df_test.head(1).index.tolist()[0]
user_ids_last = df_test.tail(1).index.tolist()[0]

plot_title="All"

In [ ]:
_ = plt.figure(figsize=(30,8))
_ = plt.title(f"Linear Regression(fitted set) | RMSE = {round(np.sqrt(mean_squared_error(df_test['y_pred'], df_test[target])),4)} | bias Error = {round(np.mean(df_test['y_pred'] - df_test[target]), 4)} | {plot_title}")
rmse_plot = plt.stem(df_test.index, df_test['y_pred'] - df_test[target], use_line_collection=True, linefmt='grey', markerfmt='D')
_ = plt.hlines(y=round(np.sqrt(mean_squared_error(df_test['y_pred'], df_test[target])),2), colors='b', linestyles='-.', label='+ RMSE', 
               xmin = user_ids_first, 
               xmax = user_ids_last
              ) 
_ = plt.hlines(y=round(-np.sqrt(mean_squared_error(df_test['y_pred'], df_test[target])),2), colors='b', linestyles='-.', label='- RMSE', 
               xmin = user_ids_first, 
               xmax = user_ids_last
              ) 
_ = plt.xticks(rotation=90, ticks=df_test.index)
_ = plt.ylabel(f"'Error = y_predicted - {target}'")
# _ = plt.ylim([(df_test['y_pred'] - df_test[grouping_var]).min(),
#               (df_test['y_pred'] - df_test[grouping_var]).max()])
_ = plt.legend()
_ = plt.show()

In [ ]:
def multiple_univariate_OLSs(X: pd.DataFrame,
                             y: pd.Series,
                            features_list: list,):
    all_coefs_df = pd.DataFrame()
    for feature in features_list:
        mod = sm.OLS(endog=y, exog=sm.add_constant(X[[feature]]))
        res = mod.fit()
        coef_df = pd.read_html(res.summary().tables[1].as_html(),header=0,index_col=0)[0].drop("const")
        coef_df = coef_df.assign(**{"rsquared": res.rsquared,
                                    "rsquared_adj": res.rsquared_adj})
        all_coefs_df = pd.concat([all_coefs_df, coef_df])
    return all_coefs_df

In [ ]:
groups_dict = {"18 - 39": ['18-29','30-39'],
              "40 - 59": ['40-49', '50-59'],
              "60+": ['60+'],
              "All": ['60+', '40-49', '18-29', '50-59', '30-39'],
              "Lower Education": 0,
              "Higher Education": 1}

In [ ]:
# tmp_ols_df[["coef", "P>|t|", "rsquared", "rsquared_adj"]]

In [ ]:
all_ols_df = pd.DataFrame()
for group in groups_dict:
    if type(groups_dict[group]) == list:
        tmp_df = df[df["demographic_age"].isin(groups_dict[group])]
    else:
        tmp_df = df[df["demographic_higher_education"] == groups_dict[group]]
        
    tmp_X = tmp_df[features_list]
    tmp_y = tmp_df[target]

    tmp_ols_df = multiple_univariate_OLSs(X=tmp_X, 
                                          y=tmp_y, 
                                          features_list=features_list)[["coef", "P>|t|", "rsquared_adj"]]
    tmp_ols_df.columns = pd.MultiIndex.from_tuples([(group, x) for x in tmp_ols_df.columns.tolist()])
    all_ols_df = pd.concat([all_ols_df, tmp_ols_df], axis=1)

In [ ]:
all_ols_df.head()

In [ ]:
_ = plt.figure(figsize=(20,10))
_ = sns.heatmap(data=all_ols_df.sort_values(by = ("All", "coef"), ascending=False),
               annot=True)
_ = plt.xlabel("")

So, the sample-level PCI is simply:

[Room for improvement] * [Weight]

Room for improvement is the distance of some centrality measure from a minimum or maximum value. The centrality measure we use in the tables in the analysis script is the mean; the minimum and maximum are the observed minimum and maximum. (you could also use e.g. scale min/max; or trimmed min/max; and you can also use median or trimmed mean etc). Whether you take the distance from the minimum or maximum depends on whether the determinant is positively or negatively associated to the criterion/target. If it's associated positively, you take the maximum (because 'increasing' the determinant increases the target), and if it's associated negatively, you take the minimum (because 'decreasing' the determinant increases the target).

The Weight is some indication of the strength of the association between the determinant and the criterion/target; e.g. the correlation or the proportion of explained variance.

The manual for the R function in the behaviorchange package is at https://r-packages.gitlab.io/behaviorchange/reference/potential_for_change.html

## All of the coefficients are positive hence the maximum is used as the extremity_measure

## because the distributions of the features and the target are scewed the median is used as the centrality_measure

### trimmed_mean = remove 2.5% up and bottom 

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from typing import Callable, Dict, Union

def apply_scaling(df: pd.DataFrame, 
                  method: Union[Callable, str] = "MinMax", 
                  kwargs: Dict = {}):
    if method == "MinMax":
        scal_df = pd.DataFrame(MinMaxScaler(**kwargs).fit_transform(df), 
             index = df.index,
            columns = df.columns)
    elif method == "Standard":
        scal_df = pd.DataFrame(StandardScaler(**kwargs).fit_transform(df), 
             index = df.index,
            columns = df.columns)
    else:
        scal_df = pd.DataFrame(method(**kwargs).fit_transform(df), 
             index = df.index,
            columns = df.columns)
    return scal_df 

In [ ]:
stats.tmean(df[features_list[0]], limits=(1, 7))

In [ ]:
# df[features_list].agg(stats.tmean)

In [ ]:
# df[features_list].agg("mean")

In [ ]:
centrality_measure = "mean"
extremity_measure = None
weight_measure = "r"

In [ ]:
print(f"The chosen parameters to calculate the PCI are:\n- centrality_measure = {centrality_measure}\n- extremity_measure = {extremity_measure}\n- weight_measure = {weight_measure}")

In [ ]:
# df[features_list].agg(extremity_measure)

In [ ]:
negative_coeffs_list = all_coefs_df[all_coefs_df["coef"] < 0].index.tolist()

In [ ]:
# all_coefs_df.loc[negative_coeffs_list, weight_measure]#.drop("const", axis=0)

In [ ]:
negative_corrs_list = corrs_df[corrs_df["r"] < 0].index.tolist()

In [ ]:
corrs_df.loc[negative_corrs_list, weight_measure]#.drop("const", axis=0)

In [ ]:
if len(negative_coeffs_list) < 0:
    
    pci_df = (
        # room for improvement calculation (series)
        (df[features_list]
         .pipe(apply_scaling)
         .agg(centrality_measure)
         - (df[features_list]
            .pipe(apply_scaling)
            .agg("max"))
        ).abs()

        *

        # weight (based on rsqaured_adj series)

        all_coefs_df[weight_measure]

    ).to_frame("PCI")
else:
    neg_pci_df = (
        # room for improvement calculation (series)
        (df[negative_coeffs_list]
         .pipe(apply_scaling)
         .agg(centrality_measure, axis=1)
         - (df[negative_coeffs_list]
            .pipe(apply_scaling)
            .agg("min"))
        ).abs()

        *

        # weight (based on rsqaured_adj series)

#         all_coefs_df.loc[negative_coeffs_list, weight_measure]
        corrs_df.loc[negative_corrs_list, weight_measure]

    ).to_frame("PCI")
    
    pos_pci_df = (
        # room for improvement calculation (series)
        (df[features_list].drop(negative_coeffs_list, axis=1)
         .pipe(apply_scaling)
         .agg(centrality_measure)
         - (df[features_list]
            .drop(negative_coeffs_list, axis=1)
            .pipe(apply_scaling)
            .agg("max"))
        ).abs()

        *

        # weight (based on rsqaured_adj series)

#         all_coefs_df[weight_measure].drop(negative_coeffs_list + ["const"], axis=0)
        corrs_df[weight_measure].drop(negative_corrs_list, axis=0)

    ).to_frame("PCI")
    
    pci_df = pd.concat([pos_pci_df, neg_pci_df])

In [ ]:
_ = plt.figure(figsize=(2, 8))
_ = sns.heatmap(data=pci_df.dropna().sort_values("PCI", ascending=False),
                annot=True
               )

In [ ]:
def potential_for_change_index(data: pd.DataFrame,
                               features_list: list,
                              centrality_measure: str = "median",
                               # extremity_measure: str = "max",
                               weight_measure: str = "rsquared_adj"):
    
    tmp_X = data[features_list]
    tmp_y = data[target]

    ols_df = multiple_univariate_OLSs(X=tmp_X, 
                                          y=tmp_y, 
                                          features_list=features_list)
    
    negative_coeffs_list = ols_df[ols_df["coef"] < 0].index.tolist()
    
    if len(negative_coeffs_list) < 0:
    
        pci_df = (
            # room for improvement calculation (series)
            (data[features_list]
             .pipe(apply_scaling)
             .agg(centrality_measure)
             - (data[features_list]
                .pipe(apply_scaling)
                .agg("max"))
            ).abs()

            *

            # weight (based on rsqaured_adj series)

#             all_coefs_df[weight_measure]
            corrs_df[weight_measure]

        ).to_frame("PCI")
    else:
        neg_pci_df = (
            # room for improvement calculation (series)
            (data[negative_coeffs_list]
             .pipe(apply_scaling)
             .agg(centrality_measure)
             - (data[negative_coeffs_list]
                .pipe(apply_scaling)
                .agg("min"))
            ).abs()

            *

            # weight (based on rsqaured_adj series)

#             all_coefs_df.loc[negative_coeffs_list, weight_measure]
            corrs_df.loc[negative_corrs_list, weight_measure]

        ).to_frame("PCI")

        pos_pci_df = (
            # room for improvement calculation (series)
            (data[features_list].drop(negative_coeffs_list, axis=1)
             .pipe(apply_scaling)
             .agg(centrality_measure)
             - (data[features_list].drop(negative_coeffs_list, axis=1)
                .pipe(apply_scaling)
                .agg("max"))
            ).abs()

            *

            # weight (based on rsqaured_adj series)

#             all_coefs_df[weight_measure].drop(negative_coeffs_list + ["const"], axis=0)
            corrs_df[weight_measure].drop(negative_corrs_list, axis=0)

        ).to_frame("PCI")

        pci_df = pd.concat([pos_pci_df, neg_pci_df])
        
    
    return pci_df

In [ ]:
bootstrap_sample_size = 1000
bootstrap_number = 100

In [ ]:
all_pcis_df = pd.DataFrame()
for i in range(0, bootstrap_number):
    # print(df.sample(n=bootstrap_sample_size, random_state=0 + i).index)
    tmp_pci_df = potential_for_change_index(data=df.sample(n=bootstrap_sample_size, random_state=0 + i),
                               features_list=features_list,
                              centrality_measure = centrality_measure,
                               # extremity_measure = "max",
                               weight_measure = weight_measure)
    all_pcis_df = pd.concat([all_pcis_df, tmp_pci_df], axis=1)

In [ ]:
all_pcis_df.columns = [f"PCI_{x}" for x in range(0, all_pcis_df.shape[1])]

In [ ]:
_ = plt.figure(figsize=(6, 8))
_ = sns.heatmap(all_pcis_df.sort_values(by="PCI_0", ascending=False))

In [ ]:
_ = plt.figure(figsize=(6, 8))
_ = sns.heatmap(all_pcis_df.agg(["min", "mean", "median", "max"], axis=1).sort_values(by="mean", ascending=False),
               annot=True,
               fmt=".3g")

In [ ]:
all_pcis_df = pd.DataFrame()
for group in groups_dict:
    if type(groups_dict[group]) == list:
        tmp_df = df[df["demographic_age"].isin(groups_dict[group])]
    else:
        tmp_df = df[df["demographic_higher_education"] == groups_dict[group]]

    tmp_pci_df = potential_for_change_index(data=tmp_df,
                               features_list=features_list,
                              centrality_measure = centrality_measure,
                               # extremity_measure = "max",
                               weight_measure = weight_measure)
    tmp_pci_df.columns = [f"PCI_{group}"]
    all_pcis_df = pd.concat([all_pcis_df, tmp_pci_df], axis=1)

In [ ]:
# all_pcis_df
_ = plt.figure(figsize=(6, 8))
_ = sns.heatmap(all_pcis_df.sort_values(by="PCI_All", ascending=False),
               annot=True,
                fmt=".3g"
               )